# Error reporting

Orange Data Mining error reports are saved on Orange server. The script below scans error report's files and collects some information. Collected data is saved to a file in CVS format.

## Settings

Please change PATH.

In [ ]:
# Please write folder to scan:
# /srv/error_report/ biolab.si server
PATH = ""

## Imports, constancs, functions

In [ ]:
import json
import fnmatch
import os

import pathlib

import pandas as pd


ADDONS = tuple(set(("Orange3-ImageAnalytics", "Orange3-Educational", "Orange3-Text", "Orange3-Prototypes",
                    "Orange3-Geo", "Orange3-Datasets", "Orange3-DataFusion", "Orange3-Recommendation",
                    "Orange-Bioinformatics", "Orange3-Network", "Orange3-Timeseries", "Orange-Infrared",
                    "Orange3-Variants", "Orange3-Textable", "Orange3-Textable-Prototypes", "Orange3-spark",
                    "tomwer")))

df_sentry = pd.DataFrame(
    columns=['datetime',
             'orange_version',
             'python',
             'os',
             'module',
             'widget_module',
             'widget_name',
             'exception',
             'machine_id',
             *ADDONS])

def scan(topdir):
    """
    Return file system paths under `topdir` that match error reports

    Parameters
    ----------
    topdir: str
        Top level directory path for the search.

    Returns
    -------
    list of paths
    """
    file_pattern = "*.txt"
    exclude_folder = "unknown"  # Omit files in that subfolder.
    paths = []

    for dirpath, dirnames, filenames in os.walk(topdir):
        for dirname in list(dirnames):
            # do not recurse into hidden dirs
            if fnmatch.fnmatch(dirname, ".*") or fnmatch.fnmatch(dirname, exclude_folder):
                dirnames.remove(dirname)
        filenames = [fname for fname in filenames if fnmatch.fnmatch(fname.lower(), file_pattern)]
        paths = paths + [os.path.join(dirpath, fname) for fname in filenames]
    return paths

## Scan direcotries and create list of files to read and analyze

In [ ]:
paths = scan(PATH)
print("Total files: ", len(paths))

## Main loop

In [ ]:
errors = []
for i, path in enumerate(paths):
    try:
        datetime = pd.to_datetime(pathlib.PurePosixPath(path).name[:-4])
        with open(path, 'r') as f:
            data = json.load(f)
        if 'Exception' in data.keys():
            exception = data['Exception'][0].split(":")[0]
        elif 'Stack Trace' in data.keys():
            exception = data['Stack Trace'][-2].split(": ")[0]
        else:
            exception = ""
        addons = ["None"] * len(ADDONS)
        if "Installed Packages" in data.keys():
            for i, a in enumerate(ADDONS):
                addons[i] = a in data["Installed Packages"]
        python, os = data['Environment'][0].split(" on ")[:2]
        orange_version = data['Version'][0].split(".dev")[0]
        os = ("Linux" in os and "Linux") or ("Windows" in os and "Windows") or ("Darwin" in os and "Darwin") or "Other"
        widget_name = data['Widget Name'][0] if 'Widget Name' in data.keys() else ""
        widget_module = data['Widget Module'][0].split(":")[0] if 'Widget Module' in data.keys() else ""
        module = data['Module'][0].split(":")[0]
        machine_id = data['Machine ID'][0]
        df_sentry.loc[df_sentry.shape[0]] = [datetime,
                                             orange_version, python, os, module, widget_module, widget_name,
                                             exception, machine_id, *addons]
    except Exception as e:
        errors.append(i)
print("Errors count: ", len(errors))

## Do not forget to save

In [ ]:
df_sentry.to_csv('errors.csv')